In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# DecisionTreeClassifier

In [2]:
#load and encode train data
df=pd.read_csv("../data/train.csv")
X=df[["Sex","Pclass","Embarked","Age"]]
y=df[["Survived"]]
encoder = LabelEncoder()
for col in ['Sex', 'Embarked', "Age"]:
    X[col] = encoder.fit_transform(X[col])

#fit model
clf=DecisionTreeClassifier()
clf.fit(X,y)

#load test X
df_test_X=pd.read_csv("../data/test.csv")
X_test=df_test_X[["Sex","Pclass","Embarked","Age"]]
encoder = LabelEncoder()
for col in ['Sex', 'Embarked', "Age"]:
    X_test[col] = encoder.fit_transform(X_test[col])

#make prediction
y_pred=clf.predict(X_test)

#load test y and ensure that this dataset maches test x
df_test_y=pd.read_csv("../data/survival.csv")
# print(df_test_X, df_test_X.shape)
# print(df_test_y, df_test_y.shape)
y_test=df_test_y["Survived"]

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

0.7200956937799043


/tmp/ipykernel_88429/2394962462.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = encoder.fit_transform(X[col])
/tmp/ipykernel_88429/2394962462.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = encoder.fit_transform(X[col])
/tmp/ipykernel_88429/2394962462.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

In [21]:
# X, y - train X and y
# X_test, y_test - test X and y

MAX_DEPTH=4

#make new model and fit it
clf2=DecisionTreeClassifier(max_depth=MAX_DEPTH)
clf2.fit(X,y)

#make prediction
y_pred=clf2.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

0.8923444976076556


In [4]:
df_test=pd.concat([X_test, y_test], axis=1)
df_test

,Sex,Pclass,Embarked,Age,Survived
0,1,3,1,44,0
1,0,3,2,60,1
2,1,2,1,74,0
3,1,3,2,34,0
4,0,3,2,27,1
...,...,...,...,...,...
413,1,3,2,79,0
414,0,1,0,51,1
415,1,3,2,50,0
416,1,3,2,79,0


In [5]:
df_men=df_test[df_test["Sex"]==1]
print(df_men["Survived"].unique())
df_women=df_test[df_test["Sex"]==0]
print(df_women["Survived"].unique())

[0]
[1]


In this case in test dataset all woman have survived and all man died so the best depth is 1 with accuracy = 1. With higher depth model is overfitted.

# Random forest classifier

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [25]:
# X, y - train X and y
# X_test, y_test - test X and y

MAX_DEPTH=4

#make new model and fit it
clf3=RandomForestClassifier(max_depth=2, random_state=0)
clf3.fit(X,y)

#make prediction
y_pred=clf3.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

/opt/anaconda/lib/python3.11/site-packages/sklearn/base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


1.0


The best depth is 2 or higher

# K Nearest Neighbors Classifier

In [8]:
from sklearn.neighbors import KNeighborsClassifier

In [33]:
# X, y - train X and y
# X_test, y_test - test X and y

#make new model and fit it
clf4=KNeighborsClassifier(n_neighbors=5)
clf4.fit(X,y)

#make prediction
y_pred=clf4.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

0.7655502392344498


/opt/anaconda/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:228: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


The best n_neighbors is 5.

# SVM with RBF kernel

In [10]:
from sklearn import svm

In [43]:
# X, y - train X and y
# X_test, y_test - test X and y

#make new model and fit it
clf5=svm.SVC(kernel="rbf", C=100, gamma=0.01)
clf5.fit(X,y)

#make prediction
y_pred=clf5.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9641148325358851


# SVM with RBF kernel GridSearchCV

In [12]:
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import classification_report

In [13]:
# X, y - train X and y
# X_test, y_test - test X and y


param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf']
}

#create svc model
svc = svm.SVC()

#perform grid search with cross-validation
grid_search = GridSearchCV(svc, param_grid, refit=True, cv=5)
grid_search.fit(X, y)

#print best parameters and estimatoe
print("Best parameters found: ", grid_search.best_params_)
print("Best estimator found: ", grid_search.best_estimator_)

#make prediction
y_pred = grid_search.predict(X_test)

# Print the classification report
print(classification_report(y_test, y_pred))

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,

Best parameters found:  {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best estimator found:  SVC(C=10, gamma=0.1)
              precision    recall  f1-score   support

           0       0.89      0.71      0.79       266
           1       0.63      0.85      0.72       152

    accuracy                           0.76       418
   macro avg       0.76      0.78      0.76       418
weighted avg       0.80      0.76      0.77       418

0.7607655502392344


/opt/anaconda/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# MLP classifier

In [14]:
from sklearn.neural_network import MLPClassifier

In [56]:
# X, y - train X and y
# X_test, y_test - test X and y

activation='relu' # relu vs tanh
solver='adam' # adam vs lbfgs

#make new model and fit it
clf7=MLPClassifier(random_state=1, max_iter=300, hidden_layer_sizes=(5,),activation=activation, solver=solver,)
clf7.fit(X,y)

#make prediction
y_pred=clf7.predict(X_test)

#calculate accuracy
accuracy= accuracy_score(y_test,y_pred)
print(accuracy)

/opt/anaconda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1102: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


1.0


/opt/anaconda/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


optimal: hidden_layer_sizes=(5,)   